https://colab.research.google.com/drive/1amt2nUo3DFm7PNSd9L9PQ1sKZBK8PDKd?usp=sharing

In [ ]:
!pip install -U sympy
# AttributeError: module 'sympy' has no attribute 'printing'

In [ ]:
!pip install optuna
!pip install catboost

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm, colors
import pandas as pd
import seaborn as sns
import os
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import (classification_report, confusion_matrix, accuracy_score,
                            f1_score, precision_score, recall_score, balanced_accuracy_score,
                            roc_auc_score)
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# PyTorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import catboost

In [ ]:
# Function to convert to one-hot vectors
def convert_to_one_hot(vector, num_classes=None):
    """
    Converts an input 1-D vector of integers into an output
    2-D array of one-hot vectors.
    """
    assert isinstance(vector, np.ndarray)
    assert len(vector) > 0

    if num_classes is None:
        num_classes = np.max(vector) + 1
    else:
        assert num_classes > 0
        assert num_classes >= np.max(vector)

    result = np.zeros(shape=(len(vector), num_classes))
    result[np.arange(len(vector)), vector] = 1
    return result.astype(int)

In [ ]:
def to_categorical(y, num_classes=None):
    """Convert class vector to one-hot matrix."""
    y = np.array(y, dtype='int')
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes), dtype=np.float32)
    categorical[np.arange(n), y] = 1
    return categorical

In [ ]:
# Custom dataset class for LSPIN
class LSPINDataset(Dataset):
    def __init__(self, data, labels, meta):
        self.data = torch.FloatTensor(data)
        self.labels = torch.FloatTensor(labels)
        self.meta = torch.FloatTensor(meta)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx], self.meta[idx]

In [ ]:
# Calculate squared distance matrix
def squared_distance(X):
    """
    Calculates the squared Euclidean distance matrix.
    X: an n-by-p tensor, which includes n samples in dimension p
    returns: n x n pairwise squared Euclidean distance matrix
    """
    r = torch.sum(X * X, 1, keepdim=True)
    D = r - 2 * torch.mm(X, X.t()) + r.t()
    return D

In [ ]:
# Gating Network
class GatingNetwork(nn.Module):
    def __init__(self, input_dim, hidden_layers, activation='tanh', stddev=0.1):
        super(GatingNetwork, self).__init__()

        self.activation_name = activation
        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'l_relu':
            self.activation = nn.LeakyReLU()
        elif activation == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        else:  # 'none'
            self.activation = nn.Identity()

        layers = []
        prev_dim = input_dim

        # Hidden layers
        for hidden_dim in hidden_layers:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(self.activation)
            prev_dim = hidden_dim

        # Output layer
        layers.append(nn.Linear(prev_dim, input_dim))
        layers.append(self.activation)

        self.layers = nn.Sequential(*layers)

        # Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0, std=stddev)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        return self.layers(x)


# Prediction Network
class PredictionNetwork(nn.Module):
    def __init__(self, input_dim, hidden_layers, output_dim, activation='relu', batch_norm=False, stddev=0.1):
        super(PredictionNetwork, self).__init__()

        self.batch_norm = batch_norm
        self.activation_name = activation

        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'l_relu':
            self.activation = nn.LeakyReLU()
        elif activation == 'sigmoid':
            self.activation = nn.Sigmoid()
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        else:  # 'none'
            self.activation = nn.Identity()

        layers = []
        prev_dim = input_dim

        # Hidden layers
        for i, hidden_dim in enumerate(hidden_layers):
            layers.append(nn.Linear(prev_dim, hidden_dim))
            if batch_norm:
                layers.append(nn.BatchNorm1d(hidden_dim))
            layers.append(self.activation)
            prev_dim = hidden_dim

        # Output layer
        self.layers = nn.Sequential(*layers)
        self.output_layer = nn.Linear(prev_dim, output_dim)

        # Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0, std=stddev)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.layers(x)
        return self.output_layer(x)


# LSPIN Model
class LSPINModel(nn.Module):
    def __init__(self,
                 input_node,
                 hidden_layers_node,
                 output_node,
                 gating_net_hidden_layers_node,
                 activation_gating='tanh',
                 activation_pred='relu',
                 feature_selection=True,
                 batch_normalization=False,
                 a=1,
                 sigma=0.5,
                 lam=0.5,
                 gamma1=0,
                 gamma2=0,
                 stddev_input=0.1,
                 stddev_input_gates=0.1):
        super(LSPINModel, self).__init__()

        # Register hyperparameters
        self.a = a
        self.sigma = sigma
        self.lam = lam
        self.gamma1 = gamma1
        self.gamma2 = gamma2
        self.feature_selection = feature_selection
        self.output_node = output_node

        # Create networks
        if feature_selection:
            self.gating_network = GatingNetwork(
                input_dim=input_node,
                hidden_layers=gating_net_hidden_layers_node,
                activation=activation_gating,
                stddev=stddev_input_gates
            )

        self.prediction_network = PredictionNetwork(
            input_dim=input_node,
            hidden_layers=hidden_layers_node,
            output_dim=output_node,
            activation=activation_pred,
            batch_norm=batch_normalization,
            stddev=stddev_input
        )

    def hard_sigmoid(self, x, a):
        """Segment-wise linear approximation of sigmoid."""
        x = a * x + 0.5
        return torch.clamp(x, min=0, max=1)

    def get_stochastic_gate(self, x, train_gates=True):
        """Compute the stochastic gates."""
        alpha = self.gating_network(x)

        if train_gates and self.training:
            # Gaussian reparametrization
            noise = torch.randn_like(x) * self.sigma
            z = alpha + noise
        else:
            z = alpha

        stochastic_gate = self.hard_sigmoid(z, self.a)
        return stochastic_gate, alpha

    def forward(self, x, compute_sim=False, train_gates=True, meta=None):
        if self.feature_selection:
            stochastic_gate, self.alpha = self.get_stochastic_gate(x, train_gates)
            x_masked = x * stochastic_gate
            self.stochastic_gates = stochastic_gate
        else:
            x_masked = x
            self.alpha = None
            self.stochastic_gates = None

        # Prediction network
        logits = self.prediction_network(x_masked)

        if self.output_node == 1:
            # Regression
            prediction = logits
        else:
            # Classification
            prediction = F.softmax(logits, dim=1)

        return prediction, logits

    def get_raw_alpha(self, x):
        """Get raw alpha values (before activation)."""
        if self.feature_selection:
            with torch.no_grad():
                self.eval()
                alpha = self.gating_network(x)
                return alpha.detach().cpu().numpy()
        else:
            return None

    def get_prob_alpha(self, x):
        """Get probability values for gates."""
        if self.feature_selection:
            alpha = self.get_raw_alpha(x)
            return self.compute_learned_prob(alpha)
        else:
            return None

    def compute_learned_prob(self, alpha):
        """Convert raw alpha to probabilities."""
        return np.minimum(1, np.maximum(0, self.a * alpha + 0.5))


# Loss function for LSPIN
class LSPINLoss(nn.Module):
    def __init__(self, lam=0.5, gamma1=0, gamma2=0, output_node=2):
        super(LSPINLoss, self).__init__()
        self.lam = lam
        self.gamma1 = gamma1
        self.gamma2 = gamma2
        self.output_node = output_node

    def forward(self, pred, y, alpha, stochastic_gates, Z=None, compute_sim=False):
        # Main prediction loss
        if self.output_node == 1:
            loss_fun = F.mse_loss(pred, y)
        else:
            loss_fun = F.cross_entropy(pred, torch.argmax(y, dim=1))

        # L0 regularization for gates
        if alpha is not None:
            # Probability of a gate being non-zero
            prob_nonzero = torch.sigmoid(alpha)
            reg_gates = self.lam * prob_nonzero.mean()

            # Similarity regularization
            if compute_sim and Z is not None:
                # Similarity matrix based on Z
                K_batch = 1.0 - squared_distance(Z) / 2.0
                # Distance matrix of gates
                D_batch = squared_distance(stochastic_gates)

                # Encourage similar samples to select similar features
                sim_term = self.gamma1 * torch.mean(K_batch * D_batch)
                # Encourage dissimilar samples to select dissimilar features
                dissim_term = self.gamma2 * torch.mean((1.0 - K_batch) * (-D_batch))

                reg_sim = sim_term + dissim_term
            else:
                reg_sim = torch.tensor(0.0, device=pred.device)

            total_loss = loss_fun + reg_gates + reg_sim
            return total_loss, loss_fun, reg_gates, reg_sim
        else:
            return loss_fun, loss_fun, torch.tensor(0.0), torch.tensor(0.0)


class LocallySparse:
    """
    Uses the LSPIN model for classification.
    """
    def __init__(self, data, n_classes):
        """
        Initialize the LocallySparse model.

        Args:
            data: raw dataframe of data
            n_classes: number of classes in the dataset
        """
        self.data = data
        self.n_classes = n_classes
        self.model = None
        self.best_model = None
        self.model_params = {}
        self.training_params = {}
        self.class_names = {}
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.trial_logs = []
        print(f"Using device: {self.device}")

        # Preprocess data
        self.x_train, self.y_train, self.x_val, self.y_val, self.x_test, self.y_test = self._preprocess_data()

    def _preprocess_data(self):
        """Preprocess data for model training."""
        db = self.data.dropna(axis=1, how='all')
        db = db.dropna(axis=0)

        db['Прогрессия'] = pd.Categorical(db['Прогрессия'])
        self.class_names = dict(enumerate(db['Прогрессия'].cat.categories))
        db['Прогрессия'] = db['Прогрессия'].cat.codes

        scaler = StandardScaler()
        y = db.pop('Прогрессия')
        y = y.values.astype(np.float32)
        y = to_categorical(y, num_classes=self.n_classes)

        x = db.values.astype(np.float32)

        x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=1)
        x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8, random_state=1)

        return (np.array(x_train), np.array(y_train),
                np.array(x_val), np.array(y_val),
                np.array(x_test), np.array(y_test))

    def _create_data_loaders(self, batch_size):
        """Create PyTorch DataLoaders for training."""
        train_dataset = LSPINDataset(self.x_train, self.y_train, self.y_train)
        val_dataset = LSPINDataset(self.x_val, self.y_val, self.y_val)
        test_dataset = LSPINDataset(self.x_test, self.y_test, self.y_test)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

        return train_loader, val_loader, test_loader

    def create_model(self, feature_selection):
        """Initialize model parameters."""
        self.model_params = {
            "input_node": self.x_train.shape[1],
            "output_node": self.n_classes,
            "feature_selection": feature_selection,
            "activation_gating": 'tanh'
        }
        self.training_params = {'batch_size': min(self.x_train.shape[0], 32)}

    def train_model(self, model, optimizer, criterion, train_loader, val_loader, epochs, compute_sim=False):
        """Train the model."""
        train_losses = []
        val_losses = []
        val_accuracies = []

        for epoch in range(epochs):
            model.train()
            epoch_loss = 0

            for x_batch, y_batch, z_batch in train_loader:
                x_batch, y_batch, z_batch = x_batch.to(self.device), y_batch.to(self.device), z_batch.to(self.device)

                # Forward pass
                predictions, logits = model(x_batch, compute_sim=compute_sim, meta=z_batch)

                # Compute loss
                loss, _, _, _ = criterion(logits, y_batch, model.alpha, model.stochastic_gates,
                                        Z=z_batch, compute_sim=compute_sim)

                # Backward pass and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                epoch_loss += loss.item()

            train_losses.append(epoch_loss / len(train_loader))

            # Validation
            if (epoch + 1) % 100 == 0: # display step
                val_loss, val_acc, _, _, _,_ = self.evaluate(model, criterion, val_loader, compute_sim)
                val_losses.append(val_loss)
                val_accuracies.append(val_acc)

                print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_losses[-1]:.6f}, "
                      f"Val Loss: {val_loss:.6f}, Val Acc: {val_acc:.6f}")

        return train_losses, val_losses, val_accuracies

    def evaluate(self, model, criterion, data_loader, compute_sim=False):
        """Evaluate the model."""
        model.eval()
        total_loss = 0
        all_preds = []
        all_targets = []

        with torch.no_grad():
            for x_batch, y_batch, z_batch in data_loader:
                x_batch, y_batch, z_batch = x_batch.to(self.device), y_batch.to(self.device), z_batch.to(self.device)

                predictions, logits = model(x_batch, compute_sim=False, train_gates=False)
                loss, _, _, _ = criterion(logits, y_batch, model.alpha, model.stochastic_gates)

                total_loss += loss.item()

                if self.n_classes > 1:  # Classification
                    preds = torch.argmax(predictions, dim=1).cpu().numpy()
                    targets = torch.argmax(y_batch, dim=1).cpu().numpy()
                else:  # Regression
                    preds = predictions.cpu().numpy()
                    targets = y_batch.cpu().numpy()

                all_preds.extend(preds)
                all_targets.extend(targets)

        # Calculate metrics
        avg_loss = total_loss / len(data_loader)

        if self.n_classes > 1:  # Classification
            matrix = confusion_matrix(all_targets, all_preds)
            accuracy = accuracy_score(all_targets, all_preds)
            f1 = f1_score(all_targets, all_preds, average='weighted')
            precision = precision_score(all_targets, all_preds, average='weighted')
            recall = recall_score(all_targets, all_preds, average='weighted')

            return avg_loss, accuracy, matrix, f1, precision, recall

    def __objective(self, trial):
        """Objective function for Optuna optimization."""
        # Define hyperparameters to optimize
        self.model_params['hidden_layers_node'] = trial.suggest_categorical(
            "hidden_layers_node",
            [[70, 20], [50, 10], [40, 20, 10], [100, 20], [50, 20, 10], [10, 10, 10], [10, 5]]
        )
        self.model_params['gating_net_hidden_layers_node'] = trial.suggest_categorical(
            "gating_net_hidden_layers_node",
            [[4, 4], [2, 2], [8], [20, 10], [2, 2, 2], [8, 2]]
        )
        self.model_params['activation_pred'] = trial.suggest_categorical(
            "activation_pred",
            ['relu', 'l_relu', 'sigmoid', 'tanh']
        )
        self.model_params['lam'] = trial.suggest_loguniform('lam', 0.005, 0.01)

        lr = trial.suggest_loguniform('learning_rate', 0.00005, 0.01)
        num_epochs = trial.suggest_categorical('num_epoch', [200, 500, 1000, 1500, 2000])

        # Create model
        model = LSPINModel(**self.model_params).to(self.device)

        print(self.model_params)

        # Define loss and optimizer
        criterion = LSPINLoss(
            lam=self.model_params['lam'],
            gamma1=0,
            gamma2=0,
            output_node=self.model_params['output_node']
        )
        optimizer = torch.optim.SGD(model.parameters(), lr=lr)

        # Create data loaders
        train_loader, val_loader, test_loader = self._create_data_loaders(
            batch_size=self.training_params['batch_size']
        )

        # Train the model
        train_losses, val_losses, val_accuracies = self.train_model(
            model, optimizer, criterion, train_loader, val_loader,
            epochs=num_epochs, compute_sim=False
        )

        # Evaluate on test set
        test_loss, test_acc, test_matrix, test_f1, test_precision, test_recall = self.evaluate(model, criterion, test_loader)
        print(f'Test Loss: {test_loss}', f'Test Accuracy: {test_acc}')
        print(f'weighted: Test F1: {test_loss}', f'Test Precision: {test_acc}', f'Test Recall: {test_acc}')
        print(test_matrix)

        trial_data = {
            'trial': trial.number,
            'train_losses': train_losses,
            'val_accuracies': val_accuracies,
            'val_losses': val_losses,
            'test_acc': test_acc,
            'test_loss': test_loss,
            'params': {
                'hidden_layers_node': self.model_params['hidden_layers_node'],
                'gating_net_hidden_layers_node': self.model_params['gating_net_hidden_layers_node'],
                'activation_pred': self.model_params['activation_pred'],
                'lam': self.model_params['lam'],
                'learning_rate': lr,
                'num_epoch': num_epochs}}

        self.trial_logs.append(trial_data)
        print(f'[Trial {trial.number}] Test acc: {test_acc}, Best val acc: {max(val_accuracies) if val_accuracies else None}')
        print('---')
        # Save model if it's the best so far
        if self.best_model is None or test_acc > self.best_acc:
            self.best_model = model
            self.best_acc = test_acc

        return test_acc

    def optimize(self, n_trials, n_jobs=1):
        """Optimize hyperparameters using Optuna."""
        self.best_acc = 0
        study = optuna.create_study(direction='maximize')
        study.optimize(self.__objective, n_trials=n_trials, n_jobs=n_jobs)

        # Get best parameters
        best_params = study.best_params
        print("Best parameters:", best_params)

        # Create data loaders
        _, _, test_loader = self._create_data_loaders(
            batch_size=len(self.x_test)
        )

        # Evaluate best model
        criterion = LSPINLoss(
            lam=best_params['lam'],
            gamma1=0,
            gamma2=0,
            output_node=self.model_params['output_node']
        )

        test_loss, test_acc = self.evaluate(self.best_model, criterion, test_loader)

        # Test predictions for metrics
        self.best_model.eval()
        with torch.no_grad():
            x_test_tensor = torch.FloatTensor(self.x_test).to(self.device)
            predictions, _ = self.best_model(x_test_tensor, train_gates=False)
            if self.n_classes > 1:
                y_pred = torch.argmax(predictions, dim=1).cpu().numpy()
                y_true = np.argmax(self.y_test, axis=1)
            else:
                y_pred = predictions.cpu().numpy()
                y_true = self.y_test

        # Calculate metrics
        accuracy = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average='weighted')
        precision = precision_score(y_true, y_pred, average='weighted')
        recall = recall_score(y_true, y_pred, average='weighted')

        print("Trial Finished*************")
        print(f"Best model's prediction architecture: {best_params['hidden_layers_node']}")
        print(f"Best model's gating architecture: {best_params['gating_net_hidden_layers_node']}")
        print(f"Best model's prediction activation function: {best_params['activation_pred']}")
        print(f"Best model's lambda: {best_params['lam']}")
        print(f"Best model's learning rate: {best_params['learning_rate']}")
        print(f"Best model's num of epochs: {best_params['num_epoch']}")
        print(f"Test accuracy: {accuracy}")
        print(f"Test F1: {f1}")
        print(f"Test precision: {precision}")
        print(f"Test recall: {recall}")

        # Save results
        results_path = "results"
        os.makedirs(results_path, exist_ok=True)
        with open(os.path.join(results_path, 'hyperparameters.txt'), "w") as file:
            file.write(
                f"prediction architecture: {best_params['hidden_layers_node']}\n"
                f"gating architecture: {best_params['gating_net_hidden_layers_node']}\n"
                f"prediction activation function: {best_params['activation_pred']}\n"
                f"lambda: {best_params['lam']}\n"
                f"learning rate: {best_params['learning_rate']}\n"
                f"number of epochs: {best_params['num_epoch']}\n"
                f"Accuracy: {accuracy}\n"
                f"F1: {f1}\n"
                f"precision: {precision}\n"
                f"recall: {recall}\n"
            )

    def get_results(self):
        """Plot the gate feature selection for each label and confusion matrix."""
        results_path = "results"
        os.makedirs(results_path, exist_ok=True)

        # Get model predictions
        self.best_model.eval()
        with torch.no_grad():
            x_test_tensor = torch.FloatTensor(self.x_test).to(self.device)
            predictions, _ = self.best_model(x_test_tensor, train_gates=False)
            if self.n_classes > 1:
                y_pred = torch.argmax(predictions, dim=1).cpu().numpy()
                y_true = np.argmax(self.y_test, axis=1)
            else:
                y_pred = predictions.cpu().numpy()
                y_true = self.y_test

        # Plot gate feature selection matrices
        test_labels = np.argmax(self.y_test, axis=1)
        gate_matrix = []

        for i in range(self.n_classes):
            label_indices = np.where(test_labels == i)[0]
            if len(label_indices) > 0:
                label_data = self.x_test[label_indices]

                # Get gate probabilities for this class
                gate_probs = self.best_model.get_prob_alpha(torch.FloatTensor(label_data).to(self.device))
                gate_matrix.append(gate_probs)

                plt.figure(figsize=(10, 8))
                sns.clustermap(gate_probs, vmin=0, vmax=1, cbar_pos=None)
                plt.xlabel('Features')
                plt.ylabel('Samples')
                plt.subplots_adjust(top=0.95, bottom=0.1, right=0.9)
                plt.suptitle(f"Label: {self.class_names[i]}", fontsize=16)
                plt.savefig(os.path.join(results_path, f"gate_matrix_{self.class_names[i]}.png"))
                plt.close()

        # Create color bar for reference
        plt.figure(figsize=(2, 4))
        sns.clustermap(np.array([[0, 0], [0, 0]]), vmin=0, vmax=1, cbar_pos=(.01, .2, .03, .4))
        plt.savefig(os.path.join(results_path, "cbar.png"))
        plt.close()

        # Plot confusion matrix
        def reverse_labels(tup):
            """Convert numerical labels to named labels."""
            return [self.class_names[x] for x in tup]

        y_true_labeled, y_pred_labeled = reverse_labels(tuple(y_true)), reverse_labels(tuple(y_pred))
        matrix = confusion_matrix(y_true_labeled, y_pred_labeled)
        df_cm = pd.DataFrame(matrix, columns=np.unique(y_true_labeled), index=np.unique(y_true_labeled))

        plt.figure(figsize=(10, 8))
        cmap = sns.cubehelix_palette(light=0.9, as_cmap=True)
        cm_normalized = df_cm.div(df_cm.sum(axis=0), axis=1)
        sns.heatmap(cm_normalized, cbar=False, annot=True, cmap=cmap, square=True, fmt='.1%', annot_kws={'size': 10})
        plt.title('LSPIN Classification')
        plt.tight_layout()
        plt.savefig(os.path.join(results_path, "lspin_results.png"))
        plt.close()

    def save_model(self):
        """Save the best model."""
        results_path = "results"
        os.makedirs(results_path, exist_ok=True)
        torch.save(self.best_model.state_dict(), os.path.join(results_path, 'model.pth'))

    def load_model(self, model_path):
        """Load a saved model."""
        model = LSPINModel(**self.model_params).to(self.device)
        model.load_state_dict(torch.load(model_path))
        model.eval()
        self.best_model = model
        return model

In [ ]:
import functools
print = functools.partial(print, flush=True)

In [ ]:
smote = SMOTE(random_state=42)
X = pd.read_pickle('https://raw.githubusercontent.com/stepikVys/gamma_knife/refs/heads/main/features_dataset_normalized.pkl')
X_train_resampled, y_train_resampled = smote.fit_resample(X.drop(columns='Прогрессия'), X['Прогрессия'])
X_test = pd.read_pickle('https://raw.githubusercontent.com/stepikVys/gamma_knife/refs/heads/main/test_dataset_normalized.pkl')

In [ ]:
data_norm = pd.concat([X_train_resampled, y_train_resampled], axis=1)
data_norm

,Число очагов,Объем очагов,Число фракций,Индекс Карновского,ОВГМ,Операция,Возраст,Время метастазирования,Время реагирования,КРР,Меланома,НМРЛ,РМЖ,РП,Таргетная терапия,Химиотерапия,Без лечения,Мужчина,Прогрессия
0,0.047619,0.177158,0.0,0.833333,1,0,0.421053,1.164881,-0.005626,0,0,0,1,0,0,0,0,0,1
1,0.380952,0.010791,0.0,0.833333,0,0,0.842105,1.143593,0.099859,0,0,0,1,0,0,1,0,0,1
2,0.095238,0.112410,0.0,0.833333,0,0,0.877193,1.000000,0.057665,0,0,0,0,1,1,0,0,1,1
3,0.047619,0.049685,0.6,0.833333,0,1,0.771930,0.712676,-0.015471,0,0,0,0,1,0,0,0,0,1
4,0.666667,0.216052,0.0,0.333333,0,0,0.508772,0.724363,0.019691,0,0,0,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,0.002905,0.056477,0.0,0.676833,0,0,0.490158,0.211182,0.088418,0,0,0,1,0,0,0,0,0,0
732,0.045841,0.061195,0.0,0.827109,0,0,0.414648,0.057650,0.011853,0,0,1,0,0,0,1,0,1,0
733,0.156686,0.228403,0.0,0.666667,0,0,0.583189,0.229784,0.000521,0,0,0,1,0,1,0,0,0,0
734,0.085760,0.232430,0.0,0.666667,0,0,0.611957,0.150926,0.008499,0,0,1,0,0,0,1,0,0,0


In [ ]:
# Check for GPU availability
def get_device():
    """
    Determines which device to use (GPU/CPU).
    :return: PyTorch device
    """
    if torch.cuda.is_available():
        print(f"GPU available: {torch.cuda.get_device_name(0)}")
        return torch.device("cuda:0")
    else:
        print("No GPU available, using CPU")
        return torch.device("cpu")

In [ ]:
device = get_device()
device

GPU available: Tesla T4


device(type='cuda', index=0)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)     # игнорировать UserWarning
warnings.filterwarnings("ignore")

In [ ]:
# Create and train the locally sparse model
device = get_device()

# Initialize the model
clf = LocallySparse(data=data_norm, n_classes=2)

# Set up model parameters
clf.create_model(feature_selection=True)

# Optimize model hyperparameters
clf.optimize(n_trials=250, n_jobs=1)

# Generate and save result visualizations
clf.get_results()

# Save the trained model
clf.save_model()

GPU available: Tesla T4
Using device: cuda


[I 2025-06-14 16:39:11,478] A new study created in memory with name: no-name-11324703-d7dc-45a6-af11-4b1917067bb7


{'input_node': 18, 'output_node': 2, 'feature_selection': True, 'activation_gating': 'tanh', 'hidden_layers_node': [100, 20], 'gating_net_hidden_layers_node': [4, 4], 'activation_pred': 'l_relu', 'lam': 0.006464328564158388}
Epoch 100/1500, Train Loss: 0.694168, Val Loss: 0.697063, Val Acc: 0.474576
Epoch 200/1500, Train Loss: 0.693256, Val Loss: 0.696274, Val Acc: 0.483051
Epoch 300/1500, Train Loss: 0.691484, Val Loss: 0.695439, Val Acc: 0.533898
Epoch 400/1500, Train Loss: 0.689521, Val Loss: 0.693858, Val Acc: 0.584746
Epoch 500/1500, Train Loss: 0.687513, Val Loss: 0.691453, Val Acc: 0.601695
Epoch 600/1500, Train Loss: 0.684123, Val Loss: 0.687905, Val Acc: 0.593220
Epoch 700/1500, Train Loss: 0.679518, Val Loss: 0.682293, Val Acc: 0.610169
Epoch 800/1500, Train Loss: 0.666967, Val Loss: 0.674148, Val Acc: 0.644068
Epoch 900/1500, Train Loss: 0.659641, Val Loss: 0.661572, Val Acc: 0.686441
Epoch 1000/1500, Train Loss: 0.629026, Val Loss: 0.642081, Val Acc: 0.720339
Epoch 1100/150

[I 2025-06-14 16:40:19,295] Trial 0 finished with value: 0.7432432432432432 and parameters: {'hidden_layers_node': [100, 20], 'gating_net_hidden_layers_node': [4, 4], 'activation_pred': 'l_relu', 'lam': 0.006464328564158388, 'learning_rate': 0.0029565552708311587, 'num_epoch': 1500}. Best is trial 0 with value: 0.7432432432432432.


{'input_node': 18, 'output_node': 2, 'feature_selection': True, 'activation_gating': 'tanh', 'hidden_layers_node': [50, 20, 10], 'gating_net_hidden_layers_node': [8, 2], 'activation_pred': 'relu', 'lam': 0.007280693634860393}
Epoch 100/500, Train Loss: 0.696754, Val Loss: 0.696195, Val Acc: 0.601695
Epoch 200/500, Train Loss: 0.696403, Val Loss: 0.695977, Val Acc: 0.618644
Epoch 300/500, Train Loss: 0.696403, Val Loss: 0.695853, Val Acc: 0.644068
Epoch 400/500, Train Loss: 0.696248, Val Loss: 0.695617, Val Acc: 0.652542
Epoch 500/500, Train Loss: 0.695758, Val Loss: 0.695334, Val Acc: 0.644068
Test Loss: 0.6968501806259155 Test Accuracy: 0.47297297297297297
weighted: Test F1: 0.6968501806259155 Test Precision: 0.47297297297297297 Test Recall: 0.47297297297297297
[[37 34]
 [44 33]]
[Trial 1] Test acc: 0.47297297297297297, Best val acc: 0.652542372881356
---


...

{'input_node': 18, 'output_node': 2, 'feature_selection': True, 'activation_gating': 'tanh', 'hidden_layers_node': [100, 20], 'gating_net_hidden_layers_node': [8, 2], 'activation_pred': 'relu', 'lam': 0.0064416858110364576}
Epoch 100/2000, Train Loss: 0.696076, Val Loss: 0.696326, Val Acc: 0.466102
Epoch 200/2000, Train Loss: 0.695130, Val Loss: 0.695911, Val Acc: 0.491525
Epoch 300/2000, Train Loss: 0.694552, Val Loss: 0.695518, Val Acc: 0.525424
Epoch 400/2000, Train Loss: 0.693072, Val Loss: 0.694987, Val Acc: 0.516949
Epoch 500/2000, Train Loss: 0.691547, Val Loss: 0.694294, Val Acc: 0.525424
Epoch 600/2000, Train Loss: 0.690716, Val Loss: 0.693381, Val Acc: 0.567797
Epoch 700/2000, Train Loss: 0.688499, Val Loss: 0.691673, Val Acc: 0.559322
Epoch 800/2000, Train Loss: 0.686287, Val Loss: 0.688931, Val Acc: 0.542373
Epoch 900/2000, Train Loss: 0.679430, Val Loss: 0.684827, Val Acc: 0.550847
Epoch 1000/2000, Train Loss: 0.670260, Val Loss: 0.678386, Val Acc: 0.576271
Epoch 1100/2000

[I 2025-06-14 19:19:16,734] Trial 139 finished with value: 0.7364864864864865 and parameters: {'hidden_layers_node': [100, 20], 'gating_net_hidden_layers_node': [8, 2], 'activation_pred': 'relu', 'lam': 0.0064416858110364576, 'learning_rate': 0.002780494914741856, 'num_epoch': 2000}. Best is trial 57 with value: 0.7905405405405406.


{'input_node': 18, 'output_node': 2, 'feature_selection': True, 'activation_gating': 'tanh', 'hidden_layers_node': [40, 20, 10], 'gating_net_hidden_layers_node': [8], 'activation_pred': 'relu', 'lam': 0.006845037963647262}
Epoch 100/2000, Train Loss: 0.696149, Val Loss: 0.696431, Val Acc: 0.525424
Epoch 200/2000, Train Loss: 0.695934, Val Loss: 0.696260, Val Acc: 0.542373
Epoch 300/2000, Train Loss: 0.695672, Val Loss: 0.696136, Val Acc: 0.525424
Epoch 400/2000, Train Loss: 0.695381, Val Loss: 0.695902, Val Acc: 0.525424
Epoch 500/2000, Train Loss: 0.694627, Val Loss: 0.695533, Val Acc: 0.584746
Epoch 600/2000, Train Loss: 0.693225, Val Loss: 0.694859, Val Acc: 0.576271
Epoch 700/2000, Train Loss: 0.691055, Val Loss: 0.693489, Val Acc: 0.567797
Epoch 800/2000, Train Loss: 0.684978, Val Loss: 0.690499, Val Acc: 0.593220
Epoch 900/2000, Train Loss: 0.677618, Val Loss: 0.681489, Val Acc: 0.652542
Epoch 1000/2000, Train Loss: 0.631336, Val Loss: 0.648074, Val Acc: 0.720339
Epoch 1100/2000,

[I 2025-06-14 19:20:38,118] Trial 140 finished with value: 0.7837837837837838 and parameters: {'hidden_layers_node': [40, 20, 10], 'gating_net_hidden_layers_node': [8], 'activation_pred': 'relu', 'lam': 0.006845037963647262, 'learning_rate': 0.005643456843915773, 'num_epoch': 2000}. Best is trial 57 with value: 0.7905405405405406.


{'input_node': 18, 'output_node': 2, 'feature_selection': True, 'activation_gating': 'tanh', 'hidden_layers_node': [40, 20, 10], 'gating_net_hidden_layers_node': [8], 'activation_pred': 'relu', 'lam': 0.0068412787476224}
Epoch 100/2000, Train Loss: 0.696541, Val Loss: 0.696296, Val Acc: 0.584746
Epoch 200/2000, Train Loss: 0.696408, Val Loss: 0.696078, Val Acc: 0.618644
Epoch 300/2000, Train Loss: 0.696245, Val Loss: 0.695912, Val Acc: 0.652542
Epoch 400/2000, Train Loss: 0.696059, Val Loss: 0.695794, Val Acc: 0.677966


In [ ]:
clf.trial_logs

In [ ]:
import json

# Сохраняем в JSON
with open("trial_logs.json", "w") as f:
    json.dump(clf.trial_logs, f, indent=4)

from google.colab import files
files.download('trial_logs.json')

In [ ]:
# Create and train the locally sparse model
device = get_device()

# Initialize the model
clf = LocallySparse(data=data_norm, n_classes=2)

# Set up model parameters
clf.create_model(feature_selection=True)

# Optimize model hyperparameters
clf.optimize(n_trials=250, n_jobs=1)

# Generate and save result visualizations
clf.get_results()

# Save the trained model
clf.save_model()

#old logs

[I 2025-06-14 15:44:34,149] A new study created in memory with name: no-name-fdc310f0-08fe-4e4a-9ba2-c38b85f19b28


GPU available: Tesla T4
Using device: cuda
{'input_node': 18, 'output_node': 2, 'feature_selection': True, 'activation_gating': 'tanh', 'hidden_layers_node': [40, 20, 10], 'gating_net_hidden_layers_node': [4, 4], 'activation_pred': 'sigmoid', 'lam': 0.0088116660810683}
Epoch 100/500, Train Loss: 0.698867, Val Loss: 0.696261, Val Acc: 0.525424
Epoch 200/500, Train Loss: 0.697818, Val Loss: 0.696661, Val Acc: 0.525424
Epoch 300/500, Train Loss: 0.697649, Val Loss: 0.697099, Val Acc: 0.525424
Epoch 400/500, Train Loss: 0.697561, Val Loss: 0.697314, Val Acc: 0.525424


[I 2025-06-14 15:45:02,582] Trial 0 finished with value: 0.4797297297297297 and parameters: {'hidden_layers_node': [40, 20, 10], 'gating_net_hidden_layers_node': [4, 4], 'activation_pred': 'sigmoid', 'lam': 0.0088116660810683, 'learning_rate': 0.00030001975479687014, 'num_epoch': 500}. Best is trial 0 with value: 0.4797297297297297.


Epoch 500/500, Train Loss: 0.697568, Val Loss: 0.697449, Val Acc: 0.525424
{'input_node': 18, 'output_node': 2, 'feature_selection': True, 'activation_gating': 'tanh', 'hidden_layers_node': [70, 20], 'gating_net_hidden_layers_node': [2, 2], 'activation_pred': 'sigmoid', 'lam': 0.009212716555321767}
Epoch 100/500, Train Loss: 0.697918, Val Loss: 0.697872, Val Acc: 0.381356
Epoch 200/500, Train Loss: 0.697882, Val Loss: 0.698051, Val Acc: 0.474576
Epoch 300/500, Train Loss: 0.697820, Val Loss: 0.697548, Val Acc: 0.525424
Epoch 400/500, Train Loss: 0.698169, Val Loss: 0.697621, Val Acc: 0.525424


[I 2025-06-14 15:45:22,879] Trial 1 finished with value: 0.5135135135135135 and parameters: {'hidden_layers_node': [70, 20], 'gating_net_hidden_layers_node': [2, 2], 'activation_pred': 'sigmoid', 'lam': 0.009212716555321767, 'learning_rate': 0.00560560635458387, 'num_epoch': 500}. Best is trial 1 with value: 0.5135135135135135.


Epoch 500/500, Train Loss: 0.697876, Val Loss: 0.697835, Val Acc: 0.474576
{'input_node': 18, 'output_node': 2, 'feature_selection': True, 'activation_gating': 'tanh', 'hidden_layers_node': [40, 20, 10], 'gating_net_hidden_layers_node': [4, 4], 'activation_pred': 'relu', 'lam': 0.0067626260483382925}
Epoch 100/1500, Train Loss: 0.696588, Val Loss: 0.696503, Val Acc: 0.474576
Epoch 200/1500, Train Loss: 0.696575, Val Loss: 0.696442, Val Acc: 0.525424
Epoch 300/1500, Train Loss: 0.696541, Val Loss: 0.696442, Val Acc: 0.533898
Epoch 400/1500, Train Loss: 0.696567, Val Loss: 0.696479, Val Acc: 0.508475
Epoch 500/1500, Train Loss: 0.696504, Val Loss: 0.696473, Val Acc: 0.533898
Epoch 600/1500, Train Loss: 0.696454, Val Loss: 0.696517, Val Acc: 0.508475
Epoch 700/1500, Train Loss: 0.696430, Val Loss: 0.696498, Val Acc: 0.508475
Epoch 800/1500, Train Loss: 0.696461, Val Loss: 0.696492, Val Acc: 0.533898
Epoch 900/1500, Train Loss: 0.696421, Val Loss: 0.696467, Val Acc: 0.542373
Epoch 1000/150

[I 2025-06-14 15:46:29,105] Trial 2 finished with value: 0.5067567567567568 and parameters: {'hidden_layers_node': [40, 20, 10], 'gating_net_hidden_layers_node': [4, 4], 'activation_pred': 'relu', 'lam': 0.0067626260483382925, 'learning_rate': 0.001767542643047017, 'num_epoch': 1500}. Best is trial 1 with value: 0.5135135135135135.


Epoch 1500/1500, Train Loss: 0.696508, Val Loss: 0.696449, Val Acc: 0.559322
{'input_node': 18, 'output_node': 2, 'feature_selection': True, 'activation_gating': 'tanh', 'hidden_layers_node': [10, 5], 'gating_net_hidden_layers_node': [8], 'activation_pred': 'relu', 'lam': 0.005892462432875085}
Epoch 100/2000, Train Loss: 0.696000, Val Loss: 0.696057, Val Acc: 0.474576
Epoch 200/2000, Train Loss: 0.696003, Val Loss: 0.696047, Val Acc: 0.483051


[W 2025-06-14 15:46:39,023] Trial 3 failed with parameters: {'hidden_layers_node': [10, 5], 'gating_net_hidden_layers_node': [8], 'activation_pred': 'relu', 'lam': 0.005892462432875085, 'learning_rate': 0.00010997743359536784, 'num_epoch': 2000} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "<ipython-input-35-1225702920>", line 429, in __objective
    _, _, _ = self.train_model(
              ^^^^^^^^^^^^^^^^^
  File "<ipython-input-35-1225702920>", line 323, in train_model
    for x_batch, y_batch, z_batch in train_loader:
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 704, in __next__
    with torch.autograd.profiler.record_function(self._profile_name):
  File "/usr/local/lib/python3.11/dist-packages/torch/autograd/profiler.py", lin

KeyboardInterrupt: 

In [ ]:
# Create and train the locally sparse model
device = get_device()

# Initialize the model
clf = LocallySparse(data=data_norm, n_classes=2)

# Set up model parameters
clf.create_model(feature_selection=True)

# Optimize model hyperparameters
clf.optimize(n_trials=250, n_jobs=1)

# Generate and save result visualizations
clf.get_results()

# Save the trained model
clf.save_model()

#old logs

[I 2025-06-14 15:27:54,301] A new study created in memory with name: no-name-9242693f-84f7-4d27-8d73-67b674c7fe76


GPU available: Tesla T4
Using device: cuda
Epoch 100/1000, Train Loss: 0.697431, Val Loss: 0.697427, Val Acc: 0.533898
Epoch 200/1000, Train Loss: 0.697398, Val Loss: 0.697432, Val Acc: 0.491525
Epoch 300/1000, Train Loss: 0.697394, Val Loss: 0.697409, Val Acc: 0.542373
Epoch 400/1000, Train Loss: 0.697395, Val Loss: 0.697405, Val Acc: 0.542373
Epoch 500/1000, Train Loss: 0.697423, Val Loss: 0.697416, Val Acc: 0.500000
Epoch 600/1000, Train Loss: 0.697349, Val Loss: 0.697386, Val Acc: 0.559322
Epoch 700/1000, Train Loss: 0.697375, Val Loss: 0.697373, Val Acc: 0.542373
Epoch 800/1000, Train Loss: 0.697360, Val Loss: 0.697400, Val Acc: 0.516949
Epoch 900/1000, Train Loss: 0.697346, Val Loss: 0.697381, Val Acc: 0.593220


[I 2025-06-14 15:28:40,055] Trial 0 finished with value: 0.4797297297297297 and parameters: {'hidden_layers_node': [10, 10, 10], 'gating_net_hidden_layers_node': [8, 2], 'activation_pred': 'l_relu', 'lam': 0.008570572123184622, 'learning_rate': 0.0012115288725070266, 'num_epoch': 1000}. Best is trial 0 with value: 0.4797297297297297.


Epoch 1000/1000, Train Loss: 0.697367, Val Loss: 0.697363, Val Acc: 0.576271
Epoch 100/2000, Train Loss: 0.694288, Val Loss: 0.695952, Val Acc: 0.533898
Epoch 200/2000, Train Loss: 0.691712, Val Loss: 0.693614, Val Acc: 0.576271
Epoch 300/2000, Train Loss: 0.686780, Val Loss: 0.690565, Val Acc: 0.635593
Epoch 400/2000, Train Loss: 0.681601, Val Loss: 0.686527, Val Acc: 0.661017
Epoch 500/2000, Train Loss: 0.674212, Val Loss: 0.680995, Val Acc: 0.677966
Epoch 600/2000, Train Loss: 0.665435, Val Loss: 0.673271, Val Acc: 0.686441
Epoch 700/2000, Train Loss: 0.652623, Val Loss: 0.662609, Val Acc: 0.677966
Epoch 800/2000, Train Loss: 0.630983, Val Loss: 0.647426, Val Acc: 0.694915
Epoch 900/2000, Train Loss: 0.613719, Val Loss: 0.627299, Val Acc: 0.703390
Epoch 1000/2000, Train Loss: 0.588473, Val Loss: 0.602228, Val Acc: 0.720339
Epoch 1100/2000, Train Loss: 0.559886, Val Loss: 0.580894, Val Acc: 0.711864
Epoch 1200/2000, Train Loss: 0.535864, Val Loss: 0.566383, Val Acc: 0.720339
Epoch 13

[I 2025-06-14 15:30:09,101] Trial 1 finished with value: 0.7297297297297297 and parameters: {'hidden_layers_node': [100, 20], 'gating_net_hidden_layers_node': [2, 2, 2], 'activation_pred': 'tanh', 'lam': 0.006088131829751003, 'learning_rate': 0.0024120189000188196, 'num_epoch': 2000}. Best is trial 1 with value: 0.7297297297297297.


...

Epoch 1500/1500, Train Loss: 0.431489, Val Loss: 0.581375, Val Acc: 0.728814
Epoch 100/1500, Train Loss: 0.687180, Val Loss: 0.690505, Val Acc: 0.618644
Epoch 200/1500, Train Loss: 0.659255, Val Loss: 0.671924, Val Acc: 0.686441
Epoch 300/1500, Train Loss: 0.584359, Val Loss: 0.616424, Val Acc: 0.728814
Epoch 400/1500, Train Loss: 0.517733, Val Loss: 0.554205, Val Acc: 0.711864
Epoch 500/1500, Train Loss: 0.473903, Val Loss: 0.545436, Val Acc: 0.711864
Epoch 600/1500, Train Loss: 0.454636, Val Loss: 0.552154, Val Acc: 0.728814
Epoch 700/1500, Train Loss: 0.435938, Val Loss: 0.555290, Val Acc: 0.720339
Epoch 800/1500, Train Loss: 0.451470, Val Loss: 0.559459, Val Acc: 0.737288
Epoch 900/1500, Train Loss: 0.431627, Val Loss: 0.563123, Val Acc: 0.728814
Epoch 1000/1500, Train Loss: 0.448157, Val Loss: 0.566474, Val Acc: 0.737288
Epoch 1100/1500, Train Loss: 0.437435, Val Loss: 0.567459, Val Acc: 0.737288
Epoch 1200/1500, Train Loss: 0.441559, Val Loss: 0.570003, Val Acc: 0.728814
Epoch 13

[I 2025-06-14 15:40:22,487] Trial 12 finished with value: 0.7297297297297297 and parameters: {'hidden_layers_node': [100, 20], 'gating_net_hidden_layers_node': [2, 2, 2], 'activation_pred': 'tanh', 'lam': 0.005052839611909299, 'learning_rate': 0.008464775375021192, 'num_epoch': 1500}. Best is trial 1 with value: 0.7297297297297297.


Epoch 1500/1500, Train Loss: 0.425425, Val Loss: 0.583897, Val Acc: 0.737288
Epoch 100/1500, Train Loss: 0.691099, Val Loss: 0.690772, Val Acc: 0.576271
Epoch 200/1500, Train Loss: 0.681774, Val Loss: 0.683357, Val Acc: 0.618644
Epoch 300/1500, Train Loss: 0.658289, Val Loss: 0.669756, Val Acc: 0.652542
Epoch 400/1500, Train Loss: 0.644577, Val Loss: 0.645698, Val Acc: 0.711864
Epoch 500/1500, Train Loss: 0.593297, Val Loss: 0.608662, Val Acc: 0.728814
Epoch 600/1500, Train Loss: 0.542519, Val Loss: 0.572245, Val Acc: 0.720339
Epoch 700/1500, Train Loss: 0.516724, Val Loss: 0.551672, Val Acc: 0.728814


[W 2025-06-14 15:40:57,113] Trial 13 failed with parameters: {'hidden_layers_node': [100, 20], 'gating_net_hidden_layers_node': [2, 2, 2], 'activation_pred': 'tanh', 'lam': 0.005725920403100939, 'learning_rate': 0.004136140592470334, 'num_epoch': 1500} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "<ipython-input-26-2909853089>", line 427, in __objective
    _, _, _ = self.train_model(
              ^^^^^^^^^^^^^^^^^
  File "<ipython-input-26-2909853089>", line 335, in train_model
    loss.backward()
  File "/usr/local/lib/python3.11/dist-packages/torch/_tensor.py", line 626, in backward
    torch.autograd.backward(
  File "/usr/local/lib/python3.11/dist-packages/torch/autograd/__init__.py", line 347, in backward
    _engine_run_backward(
  File "/usr/local/lib/python3.11/

KeyboardInterrupt: 

In [ ]:
if X_test is not None:
        print("Evaluating on test dataset...")

        # Prepare test data
        if 'Прогрессия' in X_test.columns:
            y_test = X_test.pop('Прогрессия')
            X_test_features = X_test
        else:
            X_test_features = X_test
            y_test = None

        # Make predictions if we have labels to compare against
        if y_test is not None:
            # Convert to PyTorch tensors
            X_test_tensor = torch.FloatTensor(X_test_features.values).to(device)

            # Get predictions
            clf.best_model.eval()
            with torch.no_grad():
                predictions, _ = clf.best_model(X_test_tensor, train_gates=False)
                y_pred = torch.argmax(predictions, dim=1).cpu().numpy()

            # Calculate and print metrics
            accuracy = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted')
            precision = precision_score(y_test, y_pred, average='weighted')
            recall = recall_score(y_test, y_pred, average='weighted')

            print(f"Test dataset results:")
            print(f"Accuracy: {accuracy:.4f}")
            print(f"F1 Score: {f1:.4f}")
            print(f"Precision: {precision:.4f}")
            print(f"Recall: {recall:.4f}")

            # Generate confusion matrix
            plt.figure(figsize=(8, 6))
            cm = confusion_matrix(y_test, y_pred)
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
            plt.title('Confusion Matrix on Test Dataset')
            plt.ylabel('True Label')
            plt.xlabel('Predicted Label')
            plt.tight_layout()
            plt.savefig(os.path.join("results", "test_confusion_matrix.png"))
            plt.close()

            # Print classification report
            print("\nClassification Report:")
            print(classification_report(y_test, y_pred))

            # Save test results
            with open(os.path.join("results", 'test_results.txt'), "w") as file:
                file.write(f"Test Accuracy: {accuracy:.4f}\n")
                file.write(f"Test F1 Score: {f1:.4f}\n")
                file.write(f"Test Precision: {precision:.4f}\n")
                file.write(f"Test Recall: {recall:.4f}\n\n")
                file.write("Classification Report:\n")
                file.write(classification_report(y_test, y_pred))